In [50]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [121]:
data = pd.read_csv('us-states.csv')

In [122]:
data.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [124]:
data.shape

(2889, 5)

In [125]:
import datetime 

In [126]:
#label encoder for dates
def scrub_date(date1):
    date_parts = date1.split('-')
    f_date = date(int(date_parts[0]), int(date_parts[1]), int(date_parts[2]))
    delta = f_date - date(2020, 1, 21)
    return delta.days

for i in range(len(data['date'])):
    data['date'][i] = scrub_date(data['date'][i])

/home/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [127]:
#label encoder for state
id = 1
state_dict = {}
for i in range(len(data['state'])):
    state = data['state'][i]
    if state not in state_dict:
        state_dict[state] = id
        id += 1
        data['state'][i] = state_dict[state]
    else:
        data['state'][i] = state_dict[state]
        

/home/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [128]:
data = data.drop('fips', axis = 1)

In [129]:
y = data.cases
X = data.drop('cases', axis = 1)

In [130]:
data

,date,state,cases,deaths
0,0,1,1,0
1,1,1,1,0
2,2,1,1,0
3,3,2,1,0
4,3,1,1,0
...,...,...,...,...
2884,93,33,11000,375
2885,93,1,12906,717
2886,93,54,981,31
2887,93,6,5052,257


In [131]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [132]:
from sklearn.ensemble import RandomForestRegressor

In [144]:
model = RandomForestRegressor(n_estimators = 100, random_state = 1)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

In [145]:
preds = model.predict(X_valid)

In [146]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_valid, preds)

In [147]:
mae

554.2934439834025

In [137]:
data.describe()

,cases,deaths
count,2889.000000,2889.00000
mean,4734.918311,190.28972
std,18762.852274,967.45956
min,0.000000,0.00000
25%,19.000000,0.00000
50%,365.000000,6.00000
75%,2331.000000,63.00000
max,263460.000000,15740.00000


In [155]:
def build_tf_model():
    model = keras.Sequential([
        layers.Dense(64, activation = 'relu', input_shape=[len(X_train.keys())]),
        layers.Dense(64, activation = 'relu'),
        #layers.Dense(64, activation = 'relu'),
        layers.Dense(1)
    ])
    
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mae', 'mse'])
    
    return model 

tf_model = build_tf_model()

model.summary()

AttributeError: 'RandomForestRegressor' object has no attribute 'summary'

In [156]:
history = tf_model.fit(X_train, y_train, epochs = 50, verbose = 1)

Train on 2166 samples
Epoch 1/50
2166/2166 [==============================] - 0s 194us/sample - loss: 341254499.9040 - mae: 4580.8555 - mse: 341254528.0000
Epoch 2/50
2166/2166 [==============================] - 0s 100us/sample - loss: 291195391.2830 - mae: 4265.3442 - mse: 291195360.0000
Epoch 3/50
2166/2166 [==============================] - 0s 94us/sample - loss: 230409062.0092 - mae: 3857.5068 - mse: 230409040.0000
Epoch 4/50
2166/2166 [==============================] - 0s 88us/sample - loss: 149631225.5734 - mae: 3318.2678 - mse: 149631248.0000
Epoch 5/50
2166/2166 [==============================] - 0s 111us/sample - loss: 64053908.1348 - mae: 2577.7310 - mse: 64053908.0000
Epoch 6/50
2166/2166 [==============================] - 0s 88us/sample - loss: 20059462.8391 - mae: 1893.5105 - mse: 20059456.0000
Epoch 7/50
2166/2166 [==============================] - 0s 95us/sample - loss: 16836443.9372 - mae: 1565.1006 - mse: 16836446.0000
Epoch 8/50
2166/2166 [============================

In [157]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,mae,mse,epoch
45,1.443357e+07,1152.911865,14433573.0,45
46,1.440695e+07,1151.919067,14406952.0,46
47,1.400557e+07,1167.274292,14005576.0,47
48,1.424057e+07,1148.814697,14240574.0,48
49,1.455147e+07,1174.426880,14551472.0,49


In [160]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [161]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std = 2)

In [164]:
plotter.plot({'Basic': history}, metric = 'mae')
plt.ylim([0,100])
plt.ylabel('MAE [Confirmed Cases]')

KeyError: 'val_mae'